In [1]:
import boto3
# Retrieve the list of existing buckets
s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  aws-emr-resources-235671948910-us-east-1
  aws-logs-235671948910-us-east-1
  privacylost
  zhan-commoncrawl


In [2]:
# Retrieve a bucket's ACL
result = s3.get_bucket_acl(Bucket='commoncrawl')
print(result)

{'ResponseMetadata': {'RequestId': 'BZM2KCKH4326JQVA', 'HostId': '1kB7SvVhYRd2xqGUTpjbXa8OuGGILV6SXjTBTok+zqtGnSDBrXt1uC1LuzZKn+qYlgMCNRVVv7A=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '1kB7SvVhYRd2xqGUTpjbXa8OuGGILV6SXjTBTok+zqtGnSDBrXt1uC1LuzZKn+qYlgMCNRVVv7A=', 'x-amz-request-id': 'BZM2KCKH4326JQVA', 'date': 'Mon, 23 Jan 2023 09:12:03 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Owner': {'DisplayName': 'sara', 'ID': 'fc504895e3c44cbe2a063212bf4f392a0c24a398910f6f535ec911c404ee8347'}, 'Grants': [{'Grantee': {'DisplayName': 'sara', 'ID': 'fc504895e3c44cbe2a063212bf4f392a0c24a398910f6f535ec911c404ee8347', 'Type': 'CanonicalUser'}, 'Permission': 'FULL_CONTROL'}]}


In [5]:
file_name = 'crawl-data/CC-MAIN-2015-14/segments/1427131298015.2/warc/CC-MAIN-20150323172138-00061-ip-10-168-14-71.ec2.internal.warc.gz'

offset = 882944732
length = 10489
offset_end = offset + length - 1
byte_range = "bytes={offset}-{end}".format(offset=offset, end=offset_end)
resp = s3.get_object(Bucket="commoncrawl", Key=file_name, Range=byte_range)["Body"]

In [7]:
from warcio.archiveiterator import ArchiveIterator
for record in ArchiveIterator(resp):
    url = record.rec_headers.get_header("WARC-Target-URI")
    text = record.content_stream().read()

In [1]:
# analysis 
import re
from validators import ValidationFailure
import validators
from selectolax.parser import HTMLParser
from urllib.parse import urlparse
import tldextract

regex = "((?<=[^a-zA-Z0-9])(?:https?\:\/\/|[a-zA-Z0-9]{1,}\.{1}|\b)(?:\w{1,}\.{1}){1,5}(?:com|org|edu|gov|uk|net|ca|de|jp|fr|au|us|ru|ch|it|nl|se|no|es|mil|iq|io|ac|ly|sm){1}(?:\/[a-zA-Z0-9]{1,})*)"

def is_string_an_url(url_string: str) -> bool:
    result = validators.url(url_string)

    if isinstance(result, ValidationFailure):
        return False

    return result


def get_domain_from_cc(url):
    """
    extract the domain from Common Crawl

    Args:
        url (_type_): _description_

    Returns:
        _type_: _description_
    """
    if not url:
        return None
    if is_string_an_url(url):
        domain = str(urlparse(url).netloc)
        domain = tldextract.extract(str(urlparse(url).netloc)).domain

    else:
        domain = None
    return domain

def get_text_selectolax(html):
    """extracting the tracker domain from html file

    Args:
        html (_type_): _description_

    Returns:
        _type_: _description_
    """
    trackers = []
    # try:
    tree = HTMLParser(html)
    if tree.body is None:
        return trackers
    for node in tree.tags("style"):
        node.decompose()

    #         找到a
    try:
        for node in tree.css("a,link,script,iframe,img"):
            text = node.text()
            # if "google-analytics" in text:
            #     trackers.append("google-analytics")
            if "href" in node.attributes:
                url = node.attributes["href"]
                # domain = get_domain_from_cc(url)
            if "onclick" in node.attributes:
                url = node.attributes['onclick']
                # if is_string_an_url(url):
                if "window.open" in url:
                    url = url.replace("window.open(","")
                    url = url.replace("+","")
                    url = url.replace(")","")
                    url = url.replace("'", "")
                    url = url.replace(' ',"")
                    url = url.split(";")[0]
                    url
                    print(url)
                
               
            # if "src" in node.attributes:
            #     url = node.attributes["src"]
            #     domain = get_domain_from_cc(url)

            #     if domain:
            #         print(domain)
                    

            # if (
            #     "type" in node.attributes
            #     and node.attributes["type"] == "text/javascript"
            # ):

            #     result = re.findall(regex, text)
            #     print(result)
    
                   
    except Exception as e:
        print(e)
    return trackers

    # except Exception as e:
    #     print(e)
    # finally:
    #     return trackers


In [4]:
text = '<script type="text/javascript">/*<![CDATA[*/var Liferay={Browser:{acceptsGzip:function(){return false},getMajorVersion:function(){return 0},getRevision:function(){return""},getVersion:function(){return""},isAir:function(){return false},isChrome:function(){return false},isFirefox:function(){return false},isGecko:function(){return false},isIe:function(){return false},isIphone:function(){return false},isLinux:function(){return false},isMac:function(){return false},isMobile:function(){return false},isMozilla:function(){return false},isOpera:function(){return false},isRtf:function(){return false},isSafari:function(){return false},isSun:function(){return false},isWap:function(){return false},isWapXhtml:function(){return false},isWebKit:function(){return false},isWindows:function(){return false},isWml:function(){return false}},ThemeDisplay:{getCompanyId:function(){return"1"},getUserId:function(){return"5"},getDoAsUserIdEncoded:function(){return""},getPlid:function(){return"23"},getLayoutId:function(){return"1"},getLayoutURL:function(){return"http://www.cmati.xunta.es/portada"},isPrivateLayout:function(){return"false"},getParentLayoutId:function(){return"0"},getScopeGroupId:function(){return"20"},isSignedIn:function(){return false},getLanguageId:function(){return"gl_ES"},isFreeformLayout:function(){return false},isStateExclusive:function(){return false},isStateMaximized:function(){return false},isStatePopUp:function(){return false},getPathContext:function(){return""},getPathImage:function(){return"/image"},getPathMain:function(){return"/c"},getPathThemeImages:function(){return"/tema-portal-cmati/images"},getPathThemeRoot:function(){return"/tema-portal-cmati/"},getURLHome:function(){return"http://www.cmati.xunta.es/web/guest"},getSessionId:function(){return"F8F304978CEF5625216063C0828B7C21.vmprd41"},getPortletSetupShowBordersDefault:function(){return false}}};var themeDisplay=Liferay.ThemeDisplay;/*]]>*/</script><script src="/web/20151217041141js_/http://www.cmati.xunta.es/html/js/barebone.jsp?browserId=other&amp;themeId=temaportadacmati_WAR_temaportalcmati&amp;colorSchemeId=01&amp;minifierType=js&amp;minifierBundleId=javascript.barebone.files&amp;t=1348207478000" type="text/javascript"></script><script type="text/javascript">/*<![CDATA[*/Liferay.currentURL="\u002f";Liferay.currentURLEncoded="%2F";jQuery(function(){Liferay.Util.addInputType();Liferay.Util.addInputFocus()});Liferay.Portlet.ready(function(b,a){Liferay.Util.addInputType(b,a);if(Liferay.Menu){new Liferay.Menu({button:".lfr-actions",context:a[0],trigger:".lfr-trigger"})}});Liferay.Portlet.list=["ipecos_opencms_portlet_INSTANCE_q17K","ipecos_opencms_portlet_INSTANCE_B1hq","ipecos_opencms_portlet_INSTANCE_9noY","RSSPortlet_WAR_RSSPortlet","ipecos_multi_opencms_portlet_INSTANCE_4FPy","ipecos_opencms_portlet_INSTANCE_t9I1","ipecos_opencms_portlet_INSTANCE_Zrt1","ipecos_opencms_portlet_INSTANCE_T2uC","caire_portada_WAR_caire"];if(jQuery.ui&&jQuery.ui.tabs){jQuery.extend(jQuery.ui.tabs.defaults,{navClass:"ui-tabs tabs",selectedClass:"current"})}jQuery(function(){if(Liferay.Dock){Liferay.Dock.init()}if(Liferay.Menu){new Liferay.Menu()}if(Liferay.Notice){Liferay.Notice.prototype.setClosing()}});/*]]>*/</script><link href="/web/20151217041141cs_/http://www.cmati.xunta.es/tema-portal-cmati/css_cmati/main.css?browserId=other&amp;minifierType=css&amp;t=1448349584000" rel="stylesheet" type="text/css"/>'
urls = re.findall(regex, text)
print(urls)
for url in urls:
    print(url,is_string_an_url(url))



['http://www.cmati.xunta.es/portada', 'http://www.cmati.xunta.es/web/guest', 'http://www.cmati.xunta.es/html/js/barebone', 'jQuery.ui.tabs.de', 'Liferay.Notice.prototype.se', 'http://www.cmati.xunta.es/tema']
http://www.cmati.xunta.es/portada True
http://www.cmati.xunta.es/web/guest True
http://www.cmati.xunta.es/html/js/barebone True
jQuery.ui.tabs.de False
Liferay.Notice.prototype.se False
http://www.cmati.xunta.es/tema True


In [6]:
is_string_an_url("https://web.archive.org/web/20151217041141/http://www.cmati.xunta.es/avisos?content=/Portal-Web/Contidos/Aviso/aviso_0296.html")

True

In [55]:
get_text_selectolax(text)

http://u.to/SGwhAg
http://u.to/DYXIBA
http://u.to/MeHABA
http://u.to/A3HRAQ
http://u.to/Do_1Ag
http://u.to/HucPAQ
http://u.to/qgIVCA
http://u.to/CSURBQ
http://u.to/SAWL
http://u.to/Z7hn
http://u.to/TgWL
http://u.to/7Jaw
http://u.to/Ov_K
http://u.to/GD5M
http://u.to/jv_K
http://u.to/n6A4AQ
http://u.to/HOt6
http://u.to/qv_K
http://u.to/rf_K
http://u.to/wP_K
http://u.to/JbTDAg
http://u.to/FDKnBA
http://u.to/49CLAw
http://u.to/HtvABA
http://u.to/6WnHAg
http://u.to/oeDDAg
http://u.to/qNvABA
http://u.to/oeDDAg
http://u.to/wkNZAg
http://u.to/wkNZAg
http://u.to/CSURBQ
http://u.to/AGsnAg
http://u.to/MeHABA
http://u.to/-dvABA
http://blog.ucoz.ro/
http://forum.ucoz.ro/
http://www.facebook.com/ucoz.servicii.web
http://twitter.com/ucoz_ro
http://faq.ucoz.ro/


[]

## analyse dataset

In [4]:
import pandas as pd
from ast import literal_eval 

# df = pd.read_csv("../parsed_cc_results_10.txt",'\t',names = ['url','cc_path'],converters={"cc_path": literal_eval})

df = pd.read_csv("../parsed_cc_results_10.txt",names = ['url','cc_path','year','offset','length'])

In [5]:
df.head()

,url,cc_path,year,offset,length
0,www.izoa.fr,crawl-data/CC-MAIN-2021-49/segments/1637964362...,2021,896784718,62792
1,www.izoa.fr,crawl-data/CC-MAIN-2016-50/segments/1480698542...,2016,529101936,27042
2,www.izoa.fr,crawl-data/CC-MAIN-2014-41/segments/1410657137...,2014,591070386,30082
3,www.izoa.fr,crawl-data/CC-MAIN-2019-51/segments/1575540551...,2019,738781671,20978
4,www.izoa.fr,crawl-data/CC-MAIN-2015-35/segments/1440644064...,2015,505830628,22798
